https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-data-ingest-adf
"The Data Factory pipeline saves the prepared data to your cloud storage (such as Azure Blob or Azure Data Lake).
Consume your prepared data in Azure Machine Learning by,

    Invoking an Azure Machine Learning pipeline from your Data Factory pipeline.
    OR
    Creating an Azure Machine Learning datastore.

- where to get experiment/model info (if neccesary):
    - 	 "Click on Use Web Service to go to the Consume tab in the Menu."
    - "Copy the values of the Primary Key and the Request-Response fields and save them in a document.""
    -
 https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-set-up-training-targets

 from here https://learn.microsoft.com/de-de/power-bi/connect-data/service-aml-integrate

# Train an ML model on Exasol Data

In this Turtorial You will ..

## Prerequisites

## Python script for training and saving model

 - explain module constraint if want to load model into exasol udf directly (https://docs.exasol.com/saas/database_concepts/udf_scripts.htm)
 - show or link to script

## prepare azure to run script

meant to run diractly in azurML studio. can also be archieved with remote connection to azureML but not here.

In [ ]:
!pip install azure-identity
!pip install azure-ai-ml==1.3.0

change subscription id (tut before?)
resource_group_name
workspace_name - make sure the one which has data from tut before

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<your subscription id>",               # change
    resource_group_name="<your resource group name>",       # change
    workspace_name="<your workspace name>",                 # change
)

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
compute_target = ws.compute_targets.get("<compute name>")   # change
ws.compute_targets.items
print(compute_target.get_status().serialize())

### write dependencies file for env

In [ ]:
#make env
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [ ]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - tensorflow
  - pip:
    - inference-schema[numpy-support]==1.3.0

In [ ]:
from azure.ai.ml.entities import Environment
custom_env_name = "AzureML-Exasol-tutorial"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for azureML tut",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

create ML job which runs script

change links to data files, explain how to get links

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "trucks_defaults_model"

job = command(
    inputs=dict(
        train_data=Input(
            type="uri_file",
            path="< link to training data file>",       # change
        test_data=Input(
            type="uri_file",
            path="< link to test data file>",           # change
        ),
        validation_data=Input(
            type="uri_file",
            path="< link to validation data file>",     # change
        ),
        learning_rate=0.25,
        registered_model_name=registered_model_name,
    ),
    code=".",  # location of source code, change if script not in same directory as this notebook
    command="python main.py --train_data ${{inputs.train_data}} --test_data ${{inputs.test_data}} --validation_data ${{inputs.validation_data}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment=pipeline_job_env,
    compute="Exa-AzureML-tutorial",
    experiment_name="train_model_azML_tut",
    display_name="train_model_azML_tut_",
)

run script, click link that shows up to see job details

In [ ]:
ml_client.create_or_update(job)

run test?
save model